# Carga y Exploración de Datos
Carga el conjunto de datos 'Individual household electric power consumption dataset' y realiza una exploración inicial de los datos.

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt

# Cargar el conjunto de datos
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip'
data = pd.read_csv(url, sep=';', parse_dates={'datetime': ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'])

# Mostrar las primeras filas del conjunto de datos
data.head()

# Información general del conjunto de datos
data.info()

# Estadísticas descriptivas del conjunto de datos
data.describe()

# Visualización de los datos faltantes
plt.figure(figsize=(10,6))
plt.title('Datos faltantes en el conjunto de datos')
plt.xlabel('Columnas')
plt.ylabel('Número de datos faltantes')
data.isnull().sum().plot(kind='bar')
plt.show()

# Visualización de las series temporales de consumo de energía
plt.figure(figsize=(15,6))
plt.plot(data['datetime'], data['Global_active_power'], label='Consumo de energía activa global')
plt.xlabel('Fecha')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Consumo de energía activa global a lo largo del tiempo')
plt.legend()
plt.show()

# Preprocesamiento de Datos
Limpia los valores faltantes, normaliza los datos y prepara los datos para su uso en modelos de series temporales.

In [ ]:
# Preprocesamiento de Datos

# Eliminar filas con valores faltantes
data.dropna(inplace=True)

# Convertir la columna 'Global_active_power' a tipo float
data['Global_active_power'] = data['Global_active_power'].astype(float)

# Normalizar los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data['Global_active_power'] = scaler.fit_transform(data['Global_active_power'].values.reshape(-1, 1))

# Preparar los datos para su uso en modelos de series temporales
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Definir la longitud de la secuencia
seq_length = 60

# Crear secuencias
X, y = create_sequences(data['Global_active_power'].values, seq_length)

# Dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrar las formas de los conjuntos de datos resultantes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Implementación del Modelo LSTM
Implementa un modelo de Red LSTM para predecir el consumo de energía eléctrica.

In [ ]:
# Importar las librerías necesarias para construir el modelo LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Definir el modelo LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 1)))
model_lstm.add(LSTM(50, return_sequences=False))
model_lstm.add(Dense(25))
model_lstm.add(Dense(1))

# Compilar el modelo
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
history_lstm = model_lstm.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluar el modelo
loss_lstm = model_lstm.evaluate(X_test, y_test)
print(f'Pérdida del modelo LSTM: {loss_lstm}')

# Realizar predicciones
predictions_lstm = model_lstm.predict(X_test)

# Desnormalizar las predicciones
predictions_lstm = scaler.inverse_transform(predictions_lstm)

# Desnormalizar los valores reales
y_test_descaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Visualizar las predicciones frente a los valores reales
plt.figure(figsize=(15,6))
plt.plot(y_test_descaled, label='Valores Reales')
plt.plot(predictions_lstm, label='Predicciones LSTM')
plt.xlabel('Tiempo')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Predicciones del modelo LSTM vs Valores Reales')
plt.legend()
plt.show()

# Implementación del Modelo GRU
Implementa un modelo de Red GRU para realizar la misma tarea de predicción.

In [ ]:
# Importar las librerías necesarias para construir el modelo GRU
from tensorflow.keras.layers import GRU

# Definir el modelo GRU
model_gru = Sequential()
model_gru.add(GRU(50, return_sequences=True, input_shape=(seq_length, 1)))
model_gru.add(GRU(50, return_sequences=False))
model_gru.add(Dense(25))
model_gru.add(Dense(1))

# Compilar el modelo
model_gru.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
history_gru = model_gru.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# Evaluar el modelo
loss_gru = model_gru.evaluate(X_test, y_test)
print(f'Pérdida del modelo GRU: {loss_gru}')

# Realizar predicciones
predictions_gru = model_gru.predict(X_test)

# Desnormalizar las predicciones
predictions_gru = scaler.inverse_transform(predictions_gru)

# Visualizar las predicciones frente a los valores reales
plt.figure(figsize=(15,6))
plt.plot(y_test_descaled, label='Valores Reales')
plt.plot(predictions_gru, label='Predicciones GRU')
plt.xlabel('Tiempo')
plt.ylabel('Consumo de energía (kilovatios)')
plt.title('Predicciones del modelo GRU vs Valores Reales')
plt.legend()
plt.show()

# Evaluación del Rendimiento de los Modelos
Evalúa el rendimiento de ambos modelos utilizando métricas como el Error Cuadrático Medio (RMSE) y el Error Absoluto Medio (MAE).

In [ ]:
# Evaluación del Rendimiento de los Modelos

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calcular RMSE y MAE para el modelo LSTM
rmse_lstm = np.sqrt(mean_squared_error(y_test_descaled, predictions_lstm))
mae_lstm = mean_absolute_error(y_test_descaled, predictions_lstm)

# Calcular RMSE y MAE para el modelo GRU
rmse_gru = np.sqrt(mean_squared_error(y_test_descaled, predictions_gru))
mae_gru = mean_absolute_error(y_test_descaled, predictions_gru)

# Imprimir las métricas de evaluación
print(f'RMSE del modelo LSTM: {rmse_lstm}')
print(f'MAE del modelo LSTM: {mae_lstm}')
print(f'RMSE del modelo GRU: {rmse_gru}')
print(f'MAE del modelo GRU: {mae_gru}')

# Crear una tabla comparativa de las métricas de evaluación
import pandas as pd

metrics = {
    'Modelo': ['LSTM', 'GRU'],
    'RMSE': [rmse_lstm, rmse_gru],
    'MAE': [mae_lstm, mae_gru]
}

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

# Visualizar las métricas de evaluación en un gráfico de barras
metrics_df.set_index('Modelo').plot(kind='bar', figsize=(10, 6))
plt.title('Comparación de Métricas de Evaluación')
plt.ylabel('Valor')
plt.xlabel('Modelo')
plt.xticks(rotation=0)
plt.show()

# Comparación de Resultados
Compara los resultados obtenidos por los modelos LSTM y GRU, analizando sus fortalezas y debilidades en este contexto específico. Incluye gráficos que visualicen las predicciones de ambos modelos y una tabla comparativa de las métricas de evaluación.

In [ ]:
# Evaluación del Rendimiento de los Modelos

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calcular RMSE y MAE para el modelo LSTM
rmse_lstm = np.sqrt(mean_squared_error(y_test_descaled, predictions_lstm))
mae_lstm = mean_absolute_error(y_test_descaled, predictions_lstm)

# Calcular RMSE y MAE para el modelo GRU
rmse_gru = np.sqrt(mean_squared_error(y_test_descaled, predictions_gru))
mae_gru = mean_absolute_error(y_test_descaled, predictions_gru)

# Imprimir las métricas de evaluación
print(f'RMSE del modelo LSTM: {rmse_lstm}')
print(f'MAE del modelo LSTM: {mae_lstm}')
print(f'RMSE del modelo GRU: {rmse_gru}')
print(f'MAE del modelo GRU: {mae_gru}')

# Crear una tabla comparativa de las métricas de evaluación
import pandas as pd

metrics = {
    'Modelo': ['LSTM', 'GRU'],
    'RMSE': [rmse_lstm, rmse_gru],
    'MAE': [mae_lstm, mae_gru]
}

metrics_df = pd.DataFrame(metrics)
print(metrics_df)

# Visualizar las métricas de evaluación en un gráfico de barras
metrics_df.set_index('Modelo').plot(kind='bar', figsize=(10, 6))
plt.title('Comparación de Métricas de Evaluación')
plt.ylabel('Valor')
plt.xlabel('Modelo')
plt.xticks(rotation=0)
plt.show()